In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [38]:
driver = webdriver.Chrome()

In [39]:
driver.get("https://www.playbill.com/production/gross?production=00000150-aea5-d936-a7fd-eef572240001")

In [40]:
col0 = driver.find_elements(by = By.CLASS_NAME, value = "col-0")
col1 = driver.find_elements(by = By.CLASS_NAME, value = "col-3")
col2 = driver.find_elements(by = By.CLASS_NAME, value = "col-4")

In [41]:
def check_elements(iter1):
    for e in iter1:
        print(e.text)
    print(len(iter1))

In [42]:
# # For checking the structuring of the elements which have been scraped
# check_elements(col0)
# check_elements(col1)
# check_elements(col2)

Conclusion: 
1. col0 starts with column name and continues with column values. Can be used as is, except that column header should be sliced out.
2. col1 starts with a column name, which is, however, a wrong label for the data, the html source for the website has repeated a class "col-1" twice. Thus, the column is actually "Avg Ticket - Top Ticket" and the text under it should be split into 2 different columns.
3. col2 is similar to col1, and the column name is also wrong, it should be - "Seats sold - Seats in theatre." The second column value is seen to remain mostly constant and might be truncated because it doesn't provide any additional insight.

In [43]:
i = 1
dfcol1 = [] 
dfcol2 = []
dfcol3 = []
dfcol4 = []
for x, y in zip(col1[1:], col2[2:]):
    l1, l2 = x.text.split('\n'), y.text.split('\n')
    dfcol1.append(l1[0])
    dfcol2.append(l1[1])
    dfcol3.append(l2[0])
    dfcol4.append(l2[1])

print(len(col0))
print(len(dfcol1))
print(len(dfcol3))
for e1, e2 in zip(dfcol1[-5:-1], dfcol2[-5:-1]):
    print(e1, e2)
for e1, e2 in zip(dfcol3[-5:-1], dfcol4[-5:-1]):
    print(e1, e2)

53
51
51
$126.63 $199.00
$134.30 $199.00
$153.37 $199.00
$118.96 $199.00
13,517 1,696
13,471 1,696
13,231 1,696
12,973 1,696


In [44]:
driver.quit()